<a href="https://colab.research.google.com/github/fernanda-mv/AnalisisdeDatosUQ/blob/main/Taller_GEIH_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Vamos a importar las librerias necesarias para el analisi de la GEIH
import pandas as pd
import numpy as np

In [5]:
#leyendo los datos de la GEIH 
df_cgsse= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/cgsse.CSV"
                      ,encoding="latin",sep=";")

df_desocupados= pd.read_csv( "https://raw.githubusercontent.com/niconomist98/DataAnalyticsUQ/refs/heads/main/Datos/GEIH2025/No%20ocupados.CSV"
                      ,encoding="latin",sep=";")    

df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
                      ,encoding="latin",sep=";")

df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"
                      ,encoding="latin",sep=";")                

/tmp/ipykernel_27234/3859890490.py:8: DtypeWarning: Columns (28,89,133,147,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ocupados= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Ocupados.CSV"
/tmp/ipykernel_27234/3859890490.py:11: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ft= pd.read_csv( "https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/GEIH2025/Fuerza%20de%20trabajo.CSV"


In [6]:
# 1. Calculando la poblacion total de colombia: 
ptc = df_cgsse["FEX_C18"].sum()
print('Población total de colombia:', int(ptc))

Población total de colombia: 52045001


In [7]:
#Poblacion en edad de trabajar 

sin_edad_trabajar = df_cgsse[df_cgsse['P6040']<15]['FEX_C18'].sum()
edad_trabajar = df_cgsse[df_cgsse['P6040']>=15]['FEX_C18'].sum() #PET


print('La poblacion total sin edad para trabajar es:',int(sin_edad_trabajar))
print('La poblacion total con edad para trabjar es:',int(edad_trabajar))

La poblacion total sin edad para trabajar es: 11391585
La poblacion total con edad para trabjar es: 40653416


In [8]:
#Calculado ahora la poblacion economicamente activa y definimos como poblacion activamente economica
#Tenemos que usar el DSI dado que en el modulo de no ocupados contiene informacion de personas que no estan buscando trabajo 
#No ocupados para este analisis son los que estan buscando trabajo activamente DSI=1 
pea = df_ocupados["FEX_C18"].sum() + df_desocupados[df_desocupados['DSI']==1]["FEX_C18"].sum()

#Tambien podriamos usar la variable FT=1 que es la fuerza de trabajo que cumple con las condiciones 
#Tiene trabajo o esta buscando trabajo

#definimos la poblacion economicamente inactiva como
#Usamos la columna FFT que son las personas fuera de la fuerza de trabajo 
pei = df_ft[df_ft['FFT']==1]['FEX_C18'].sum()

print(f'La poblacion economicamente activa es:', int(pea))
print(f'La poblacion economicamente inactiva es:', int(pei))

La poblacion economicamente activa es: 25974846
La poblacion economicamente inactiva es: 14678569


In [9]:
# Ahora vamos a calcular la tasa de ocupacion (TO)
TO = (df_ocupados['FEX_C18'].sum() / edad_trabajar)

print(f'La tasa de ocupacion: {TO:.2%}')

#Calculando el desempleo (TD) tenemos que

TD = (df_desocupados[df_desocupados['DSI']==1]['FEX_C18'].sum() / pea)
print(f'La tasa de desempleo es {TD:.2%}')

La tasa de ocupacion: 58.43%
La tasa de desempleo es 8.55%


In [18]:
#Calcule las tasas de desempleo para hombres y mujeres en todos los departamentos de colombia,
#cuales son los departamentos con mayores y menores tasas de desempleo ? 
# Hacemos merge para tener el sexo en el dataset de desocupados
df_desocupados_gen = pd.merge(
    df_desocupados[df_desocupados["DSI"]==1], 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left'
)
# Agrupamos por departamento y sexo
desocupados_sexo = (
    df_desocupados_gen.groupby(['DPTO','P3271'])['FEX_C18']
    .sum()
    .reset_index()
    .rename(columns={'FEX_C18':'desocupados'})
)

# Ocupados también con sexo
df_ocupados_gen = pd.merge(
    df_ocupados, 
    df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']], 
    on=['DIRECTORIO','SECUENCIA_P','ORDEN'], 
    how='left'
)

ocupados_sexo = (
    df_ocupados_gen.groupby(['DPTO','P3271'])['FEX_C18']
    .sum()
    .reset_index()
    .rename(columns={'FEX_C18':'ocupados'})
)

# Unimos ocupados y desocupados para calcular la PEA
pea_sexo = pd.merge(ocupados_sexo, desocupados_sexo, on=['DPTO','P3271'], how='outer').fillna(0)
pea_sexo['PEA'] = pea_sexo['ocupados'] + pea_sexo['desocupados']

# Calculamos la tasa de desempleo (TD)
pea_sexo['TD'] = pea_sexo['desocupados'] / pea_sexo['PEA'] * 100

# Reemplazamos sexo por etiquetas
pea_sexo['Sexo'] = pea_sexo['P3271'].map({1:'Hombres', 2:'Mujeres'})

# Mostramos la tabla final
print(pea_sexo[['DPTO','Sexo','TD']].head(20))

    DPTO     Sexo         TD
0      5  Hombres   4.886691
1      5  Mujeres   9.770415
2      8  Hombres   6.301949
3      8  Mujeres  10.232567
4     11  Hombres   9.821256
5     11  Mujeres   7.078481
6     13  Hombres   4.791098
7     13  Mujeres  12.028807
8     15  Hombres   6.967972
9     15  Mujeres   4.008697
10    17  Hombres   5.583183
11    17  Mujeres   7.677962
12    18  Hombres   2.676220
13    18  Mujeres  15.726909
14    19  Hombres   3.390661
15    19  Mujeres   5.146277
16    20  Hombres   5.991357
17    20  Mujeres  13.209651
18    23  Hombres   5.832048
19    23  Mujeres  19.837285


In [19]:
# Mayor y menor TD por sexo
td_max = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmax()]
td_min = pea_sexo.loc[pea_sexo.groupby("Sexo")["TD"].idxmin()]

print("Departamentos con MAYOR tasa de desempleo por sexo:")
print(td_max[['DPTO','Sexo','TD']])

print("\nDepartamentos con MENOR tasa de desempleo por sexo:")
print(td_min[['DPTO','Sexo','TD']])

Departamentos con MAYOR tasa de desempleo por sexo:
    DPTO     Sexo         TD
48    81  Hombres  18.057931
49    81  Mujeres  31.660892

Departamentos con MENOR tasa de desempleo por sexo:
    DPTO     Sexo        TD
56    91  Hombres  1.979606
57    91  Mujeres  1.874288


In [ ]:
# podemos ver que el departamento con mayor desempleo lo que va en el 2025 es el 81 que corresponde a Arauca
# y con menor desempleo es 91 que es Amazonas 

In [ ]:
#Como podemos ver esta columna de genero no existe, entonces hacemos un merge para agregar nuestras columnas
#Entonces vamos a separar entre hombres y mujeres 
#Vamos a calcular la peam y peah

df_desocupados_ec = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==63) | (df_desocupados['DPTO']==66) (df_desocupados['DPTO']==17)]
PQRC = df_desocupados[(df_desocupados['DSI']==1) & (df_desocupados['DPTO']==63) | (df_desocupados['DPTO']==66) |(df_desocupados['DPTO']==17)]['FEX_C18'].sum()
print(f' POBLACION DEL QUINDIO, RISARALDA y CALDAS:',int(PQRC))

 POBLACION DEL QUINDIO, RISARALDA y CALDAS: 778290


In [12]:
#Ahora vamos a separarlos por genero

df_desocupados_genero_ec = pd.merge(df_desocupados_ec, df_cgsse[['DIRECTORIO','SECUENCIA_P','ORDEN','P3271']],
                       on=['DIRECTORIO','SECUENCIA_P','ORDEN'], how='left')

PHDO = df_desocupados_genero_ec[df_desocupados_genero_ec['P3271']==1]['FEX_C18'].sum()
PMDO = df_desocupados_genero_ec[df_desocupados_genero_ec['P3271']==2]['FEX_C18'].sum()
print(f' Hombres desempleados:',int(PHDO))
print(f' Mujeres desempleados:',int(PMDO))

 Hombres desempleados: 262748
 Mujeres desempleados: 515542
